In [1]:
from imp import reload
from hitbtc import HitBTC
import model

import pandas as pd
import torch
import json

#_ = reload(model), reload(hitbtc)

In [2]:
class Trader(HitBTC):
    def __init__(self, key, secret, pair):
        self.pair = pair
        super(Trader, self).__init__(key, secret)
        self.auth()
        self.send('subscribeTrades', {'symbol': pair}, 0)
        self.lstm = model.get_model()
        self.num_correct = 1
        self.num_incorrect = 0
        self.next_prediction = None
        self.last_value = None
        self.error_diff_long = 0
        self.error_diff_short = 0
    
    def handle_messages(self, message):
        print(f'correct: {self.num_correct}, incorrect: {self.num_incorrect}: {self.num_correct * 100 / (self.num_incorrect + self.num_correct)}%, error long: {trader.error_diff_long}, error short: {trader.error_diff_short}')
        new_value = float(json.loads(self.messages[-1])['params']['data'][0]['price'])
        if self.next_prediction is not None and self.last_value is not None:
            if new_value > self.last_value:
                if self.next_prediction == 1:
                    self.num_correct += 1
                else:
                    self.num_incorrect += 1
                    self.error_diff_long += abs(new_value - self.last_value)
            elif new_value < self.last_value:
                if self.next_prediction == 0:
                    self.num_correct += 1
                else:
                    self.num_incorrect += 1
                    self.error_diff_short += abs(new_value - self.last_value)
        self.last_value = new_value
            
        if len(self.messages) == self.max_size:
            try:
                df = pd.DataFrame([a for b in [json.loads(m)['params']['data'] for m in self.messages] for a in b])
                df = df.assign(price=lambda x: pd.to_numeric(x.price)).assign(relative_returns=lambda x: (x.price - x.price.shift(1)) / x.price)[-10:]
                inputs = torch.from_numpy(pd.np.expand_dims(df.relative_returns.values, 1)[None]).float()
                predictions = self.lstm(inputs)
                prediction = torch.max(predictions, 1)[1][0].item()
                self.next_prediction = prediction
            except Exception as e:
                print(e)

In [3]:
key = 'd4e97b7f4dda4163fff6860f4db03646'
secret = 'f59e82ee2d2bd4506dc9906422097db5'

client = HitBTC(key, secret)
trader = Trader(key, secret, 'BTCUSD')

correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error 

In [4]:
1

1

correct: 2832, incorrect: 2681: 51.36949029566479%, error long: 1528.0599999999858, error short: 2948.130000000001
correct: 2832, incorrect: 2682: 51.36017410228509%, error long: 1529.4599999999855, error short: 2948.130000000001
correct: 2833, incorrect: 2682: 51.3689936536718%, error long: 1529.4599999999855, error short: 2948.130000000001
correct: 2833, incorrect: 2683: 51.359680928208846%, error long: 1531.6599999999853, error short: 2948.130000000001
correct: 2834, incorrect: 2683: 51.368497371760014%, error long: 1531.6599999999853, error short: 2948.130000000001
correct: 2834, incorrect: 2684: 51.35918811163465%, error long: 1531.6599999999853, error short: 2949.9000000000005
correct: 2834, incorrect: 2685: 51.349882225040766%, error long: 1532.689999999985, error short: 2949.9000000000005
correct: 2834, incorrect: 2686: 51.34057971014493%, error long: 1532.689999999985, error short: 2952.2000000000007
correct: 2835, incorrect: 2686: 51.34939322586488%, error long: 1532.68999999

In [12]:
#client.send('unsubscribeTicker', {'symbol': 'ETHBTC'})
#client.send('unsubscribeTrades', {'symbol': 'ETHBTC'})

In [18]:
client.send('getTradingBalance', {}, 1)

In [19]:
pd.DataFrame([json.loads(m) for m in client.messages])

,id,jsonrpc,result
0,15276309459475,2.0,True
1,1,2.0,"[{'currency': '1ST', 'available': '0', 'reserv..."
